# KPI : Pourcentage de personnes qui obtiennent un résultat conventionnel dans les pages personnalisables (contributions)

**Définition** : Part des personnes qui sélectionnent une CC traitée dans les pages personnalisables par rapport au nombre total de personnes qui consultent des pages personnalisables

**Formule** : (Nb de visites sur contributions + CC select dans les contributions)/(Nb de visites sur contributions)*100

In [ ]:
import pandas as pd
import time
import json
import math
#from tqdm.notebook import tqdm
#import plotly.express as px
#import plotly.graph_objects as go
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
from query_elastic_search import load_file_if_exists_or_execute_query

In [ ]:
QUERY = {
    "query": {
        "bool": { 
          "must": [
            {
              "prefix": {
                "url": "https://code.travail.gouv.fr/contribution/" 
              }
            },
     #       {
     #         "range": {
     #          "lastActionDateTime": {
     #             "gte": "2022-04-01 00:00:00",
     #             "lt": "2022-05-01 00:00:00"
     #          }
     #         }
     #       },
            {
              "range": {
               "logfile": {
                  "gte": "2022-04-01",
                  "lt": "2022-05-01"
               }
              }
            }
          ]
        }
      }
}

In [ ]:
logs_contrib = load_file_if_exists_or_execute_query("", QUERY)

## on peut utiliser ce jeu de données si déjà DL via la commande retrieve de monolog
from join_multiple_file import pd_read_pattern, PATH_CDTN_MONOLOG

logs_contrib = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-april/*")
logs_contrib = logs_contrib[
    logs_contrib['url'].str.match('^https://code.travail.gouv.fr/contribution/', 
                                                     na=False)
].copy()

In [ ]:
logs_contrib.shape

In [ ]:
logs_contrib.head()

In [ ]:
logs_contrib['url'] = logs_contrib['url'].str.split('#').str[0]

In [ ]:
logs_contrib_without_duplicates = logs_contrib.drop_duplicates(subset=['idVisit', 'url', 'type']).copy()

In [ ]:
logs_contrib_without_duplicates.shape[0]

# 3.1
Définition : Part des personnes qui sélectionnent une CC dans les pages personnalisables par rapport au nombre total de personnes qui consultent des pages personnalisables (en excluant celles qui ont un n°IDCC dans l'url)

Formule : CC select dans les contributions/Nb de visites sur contributions (en excluant celles qui ont un n°IDCC dans l'url)*100

### On commence par le dénominateur : les visites sur les pages contribution sans IDCC dans l'url

In [ ]:
visits_on_contrib_without_idcc = logs_contrib_without_duplicates[
    logs_contrib_without_duplicates['url'].str.match('^https://code.travail.gouv.fr/contribution/[a-zA-Z]+', 
                                                     na=False)
]

In [ ]:
nb_total_visits_on_contrib_without_idcc = visits_on_contrib_without_idcc.type.value_counts()['visit_content']
nb_total_visits_on_contrib_without_idcc

### Numérateur : CC select dans les contributions

In [ ]:
cc_select_on_contrib_without_idcc = visits_on_contrib_without_idcc.type.value_counts()['cc_select']
cc_select_on_contrib_without_idcc

### Calcul du Pourcentage de personnes qui selectionne une CC dans les pages non personnalisées

In [ ]:
print(f"{round(( cc_select_on_contrib_without_idcc) / nb_total_visits_on_contrib_without_idcc * 100 , 2)} %")

# 3.2 
Définition : Part des personnes qui consultent une pages personnalisable personnalisée (soit par un CC select soit en arrivant direct sur une page personnalisée avec une recherche internet) par rapport au nombre total de personnes qui consultent des pages personnalisables (= toutes les url qui commencent par "/contribution" = en incluant celles qui ont un n°IDCC dans l'url)

Formule : (Somme des visites de url qui commencent par chiffre + CC select dans les pages contribution)/Somme des visites sur toutes les url qui commencent par "/contribution"

### On va commencer par trouver les visites sur les pages contribution (dénominateur)

In [ ]:
nb_total_visits_on_contrib = logs_contrib_without_duplicates.type.value_counts()['visit_content']
nb_total_visits_on_contrib

### Comptons le nombre de visites sur les pages personnalisées 

In [ ]:
logs_on_cc_contrib = logs_contrib_without_duplicates[
    (logs_contrib_without_duplicates['url'].str.match('^https://code.travail.gouv.fr/contribution/[0-9]+', na=False)) &
    (logs_contrib_without_duplicates['type']=="visit_content")
].copy()
logs_on_cc_contrib.head()


In [ ]:
nb_cc_contrib = logs_on_cc_contrib.shape[0]
nb_cc_contrib

### Comptons le nombre de cc_select sur les pages contrib

In [ ]:
nb_cc_select_on_contrib = logs_contrib_without_duplicates.type.value_counts()['cc_select']
nb_cc_select_on_contrib

## Calcul du Pourcentage de personnes qui obtiennent un résultat conventionnel dans les pages personnalisables 

In [ ]:
round((nb_cc_contrib + nb_cc_select_on_contrib) / nb_total_visits_on_contrib * 100 , 2)

In [ ]:
round(( nb_cc_select_on_contrib) / nb_total_visits_on_contrib * 100 , 2)

In [ ]:
round((nb_cc_contrib ) / nb_total_visits_on_contrib * 100 , 2)